In [15]:
from tensorflow.keras.models import load_model
from qkeras.utils import _add_supported_quantized_objects

co = {}
_add_supported_quantized_objects(co)

import os
os.environ['PATH'] = os.environ['XILINX_VIVADO'] + '/bin:' + os.environ['PATH']
os.environ['LD_PRELOAD'] = '/lib/x86_64-linux-gnu/libudev.so.1'

In [2]:
import tensorflow as tf

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

cifar10_classes=['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']

In [16]:
#model = load_model('AlexNet_Weights/AlexNet_Normal_Model.h5')
model = load_model('AlexNet_Weights/AlexNet_check_best_model.h5', custom_objects=co)
model.summary()

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1 (QConv2D)             (None, 28, 28, 5)         380       
                                                                 
 relu1 (QActivation)         (None, 28, 28, 5)         0         
                                                                 
 pool1 (MaxPooling2D)        (None, 13, 13, 5)         0         
                                                                 
 conv2 (QConv2D)             (None, 9, 9, 13)          1638      
                                                                 
 relu2 (QActivation)         (None, 9, 9, 13)          0         
                                                                 
 pool2 (MaxPooling2D)        (None, 4, 4, 13)          0         
                                                                 
 conv3 (QConv2D)             (None, 4, 4, 20)         

In [19]:
import hls4ml
import plotting

config = hls4ml.utils.config_from_keras_model(model, granularity='name')

config['Model']['Precision'] = 'ap_fixed<16,6>'
config['Model']['ReuseFactor'] = 1

for Layer in config['LayerName'].keys():
    config['LayerName'][Layer]['Strategy'] = 'Latency'
    config['LayerName'][Layer]['ReuseFactor'] = 1
    #config['LayerName'][Layer]['Precision'] = 'ap_fixed<8,4>'

config['LayerName']['softmax']['Strategy'] = 'Stable'
'''
for layer in ['conv1', 'conv2'] :
    config['LayerName'][layer]['Precision'] = 'ap_fixed<8,4>'
'''
print("-----------------------------------")
plotting.print_dict(config)
print("-----------------------------------")

cfg = hls4ml.converters.create_config(backend='VivadoAccelerator')
cfg['IOType'] = 'io_stream'
cfg['HLSConfig'] = config
cfg['KerasModel'] = model
cfg['OutputDir'] = 'AlexNet_ZCU104'
cfg['Board'] = 'zcu104'

hls_model = hls4ml.converters.keras_to_hls(cfg)

hls_model.compile()

Interpreting Sequential
Topology:
Layer name: input_5, layer type: InputLayer, input shapes: [[None, 32, 32, 3]], output shape: [None, 32, 32, 3]
Layer name: conv1, layer type: QConv2D, input shapes: [[None, 32, 32, 3]], output shape: [None, 28, 28, 5]
Layer name: relu1, layer type: Activation, input shapes: [[None, 28, 28, 5]], output shape: [None, 28, 28, 5]
Layer name: pool1, layer type: MaxPooling2D, input shapes: [[None, 28, 28, 5]], output shape: [None, 13, 13, 5]
Layer name: conv2, layer type: QConv2D, input shapes: [[None, 13, 13, 5]], output shape: [None, 9, 9, 13]
Layer name: relu2, layer type: Activation, input shapes: [[None, 9, 9, 13]], output shape: [None, 9, 9, 13]
Layer name: pool2, layer type: MaxPooling2D, input shapes: [[None, 9, 9, 13]], output shape: [None, 4, 4, 13]
Layer name: conv3, layer type: QConv2D, input shapes: [[None, 4, 4, 13]], output shape: [None, 4, 4, 20]
Layer name: relu3, layer type: Activation, input shapes: [[None, 4, 4, 20]], output shape: [None

Done


In [20]:
hls_model.build(csim=True, export=True, bitfile=True)


****** Vivado(TM) HLS - High-Level Synthesis from C, C++ and SystemC v2019.2 (64-bit)
  **** SW Build 2708876 on Wed Nov  6 21:39:14 MST 2019
  **** IP Build 2700528 on Thu Nov  7 00:09:20 MST 2019
    ** Copyright 1986-2019 Xilinx, Inc. All Rights Reserved.

source /opt/Xilinx/Vivado/2019.2/scripts/vivado_hls/hls.tcl -notrace
INFO: Applying HLS Y2K22 patch v1.2 for IP revision
INFO: [HLS 200-10] Running '/opt/Xilinx/Vivado/2019.2/bin/unwrapped/lnx64.o/vivado_hls'
INFO: [HLS 200-10] For user 'jovyan' on host '6307b0c947c6' (Linux_x86_64 version 4.15.0-212-generic) on Fri Jul 07 04:55:02 UTC 2023
INFO: [HLS 200-10] In directory '/home/jovyan/Internship_Waseda/hls4ml/AlexNet/AlexNet_ZCU104'
Sourcing Tcl script 'build_prj.tcl'
INFO: [HLS 200-10] Opening project '/home/jovyan/Internship_Waseda/hls4ml/AlexNet/AlexNet_ZCU104/myproject_prj'.
INFO: [HLS 200-10] Adding design file 'firmware/myproject_axi.cpp' to the project
INFO: [HLS 200-10] Adding design file 'firmware/myproject.cpp' to the 

ERROR: [BD 5-390] IP definition not found for VLNV: xilinx.com:hls:myproject_axi:1.0 


CSynthesis report not found.
Vivado synthesis report not found.
Cosim report not found.
Timing report not found.


{'CSimResults': []}